# RNN을 사용한 주가 예측

1985.01.04 ~ 2020.01.17

25년 데이터 예측 모델을 통해 

2020.01.09 ~ 2020.02.28 종가 예측

rmse 확인하기

In [78]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [79]:
import keras.backend.tensorflow_backend as Keras_GPU

def get_session():
    device_count = {'CPU':1,'GPU':0}
    return tf.Session(config=tf.ConfigProto(device_count = {'CPU' : 1,'GPU' : 0}))
Keras_GPU.set_session(get_session())

In [80]:
def MinMaxScaler(data):
    denom=np.max(data,0)-np.min(data,0)
    nume=data-np.min(data,0)
    return nume/denom    
    #정규화

In [122]:
path = r"Q:\Python_Data\dataset"
name = "\\samsung_stock_price.csv"

In [123]:
data = pd.read_csv(path+name, encoding='ANSI')
# data = np.loadtxt(path+name, delimiter=',')

In [124]:
data.drop(['일자'], axis=1, inplace=True)

In [125]:
data.head()

,시가,고가,저가,종가,거래량
0,55700,56400,55600,55800,9893846
1,54900,55600,54600,55500,10278951
2,56000,56600,54900,55500,15422255
3,55500,56000,55000,55200,12993228
4,56200,56600,55700,55800,8356767


In [126]:
data = np.array(data)

In [127]:
data = data[::-1]

In [128]:
trainSize = int(len(data)*0.7)
seqLength = 10

trainSet = data[:trainSize]
testSet = data[trainSize-seqLength:]

In [129]:
def MinMaxScaler(data):
    denom=np.max(data,0)-np.min(data,0)
    nume=data-np.min(data,0)
    return nume/denom    
    #정규화

trainSet=MinMaxScaler(trainSet)
testSet=MinMaxScaler(testSet)

In [130]:
tf.reset_default_graph()

In [131]:
targetLength = 1

In [132]:
trainSet[:5]

array([[0.00092814, 0.00092324, 0.00094181, 0.00085729, 0.00034218],
       [0.00086184, 0.00085729, 0.00087454, 0.00079135, 0.00033218],
       [0.00086184, 0.00092324, 0.00087454, 0.00085729, 0.00236327],
       [0.00086184, 0.00085729, 0.00080727, 0.0007254 , 0.00258739],
       [0.00066295, 0.00065946, 0.0004709 , 0.00046162, 0.00099453]])

In [133]:
targetLength = 2

In [134]:
def buildDataSet(timeSeries, seqLength):
    xdata=[]
    ydata=[]
    for i in range(0, len(timeSeries)-seqLength-targetLength):
        tx=timeSeries[i:i+seqLength,[0,1,2,4]]
        ty=timeSeries[i+seqLength:i+seqLength+targetLength,[3]]
        xdata.append(tx)
        ydata.append(ty)
    print(tx.shape, ty.shape)
    return np.array(xdata), np.array(ydata)

In [135]:
trainX, trainY=buildDataSet(trainSet, seqLength)
testX, testY=buildDataSet(testSet, seqLength)

(10, 4) (2, 1)
(10, 4) (2, 1)


In [136]:
print(trainX.shape, trainY.shape)
print(testX.shape, testY.shape)

(6489, 10, 4) (6489, 2, 1)
(2785, 10, 4) (2785, 2, 1)


In [137]:
trainY[0]
trainX[0]

array([[0.00092814, 0.00092324, 0.00094181, 0.00034218],
       [0.00086184, 0.00085729, 0.00087454, 0.00033218],
       [0.00086184, 0.00092324, 0.00087454, 0.00236327],
       [0.00086184, 0.00085729, 0.00080727, 0.00258739],
       [0.00066295, 0.00065946, 0.0004709 , 0.00099453],
       [0.00053036, 0.00059351, 0.00060545, 0.00141676],
       [0.00053036, 0.00059351, 0.00060545, 0.00121865],
       [0.00072925, 0.0007254 , 0.00073999, 0.00157284],
       [0.00072925, 0.0007254 , 0.00067272, 0.00066035],
       [0.00066295, 0.0007254 , 0.00073999, 0.00161286]])

In [138]:
dataDim = 4

x=tf.placeholder(tf.float32, [None, seqLength, dataDim])
y=tf.placeholder(tf.float32, [None, targetLength, 1])

In [139]:
hiddenDim = 100

In [140]:
cell=tf.contrib.rnn.BasicLSTMCell(
    num_units=hiddenDim, 
    state_is_tuple=True,
    activation=tf.tanh
    #([1,2],[3,4])
    #is_Tuple=True => 튜플 = ([1,2],[3,4])
    #         False => 텐서 = [1,2,3,4]
)

In [146]:
outputDim=2
lr=0.01
iterations=500

In [147]:
outputs, _states=tf.nn.dynamic_rnn(cell, x, dtype=tf.float32)
yhat=tf.contrib.layers.fully_connected(outputs[:,-2], outputDim, activation_fn=None)

In [150]:
loss= tf.reduce_mean(tf.square(yhat-y))
opt=tf.train.AdamOptimizer(lr)
train=opt.minimize(loss)

In [121]:
#rmse
targets=tf.placeholder(tf.float32, [None,1])
predictions=tf.placeholder(tf.float32, [None,1])
rmse=tf.sqrt(tf.reduce_mean(tf.square(targets-predictions)))

In [116]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(iterations):
        _,cv=sess.run([train, loss], feed_dict={
            x:trainX, y:trainY
        })
        print("step:{} loss:{}".format(i, cv))
    testPredict=sess.run(yhat, feed_dict={x:testX})
    rmsev=sess.run(rmse, feed_dict={
        targets:testY,
        predictions:testPredict
    })
    print("rmse value: {}".format(rmsev))

InvalidArgumentError: 2 root error(s) found.
  (0) Invalid argument: Incompatible shapes: [6489,1] vs. [6489,2,1]
	 [[node sub (defined at C:\Users\chan\AppData\Roaming\Python\Python37\site-packages\tensorflow_core\python\framework\ops.py:1748) ]]
	 [[Mean/_13]]
  (1) Invalid argument: Incompatible shapes: [6489,1] vs. [6489,2,1]
	 [[node sub (defined at C:\Users\chan\AppData\Roaming\Python\Python37\site-packages\tensorflow_core\python\framework\ops.py:1748) ]]
0 successful operations.
0 derived errors ignored.

Original stack trace for 'sub':
  File "C:\Users\chan\anaconda3\envs\tf15_py37\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\chan\anaconda3\envs\tf15_py37\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\chan\anaconda3\envs\tf15_py37\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\chan\anaconda3\envs\tf15_py37\lib\site-packages\traitlets\co

In [ ]:
plt.figure(figsize=(100,5))
plt.plot(testY)
plt.plot(testPredict)
plt.show()